In [473]:
pip install redis geopandas folium  geopy --index-url https://pypi.org/simple


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [496]:
import redis
import random
import time
import geopandas as gpd
import folium
from shapely.geometry import Point
import geopy.distance

In [497]:
# Start Redis

# Connect to Redis 
redis_client = redis.Redis(host='localhost', port=6379)


In [498]:
# Let's do a clean up 
redis_client.delete("available_drivers")
redis_client.delete("rider_queries")

0

In [499]:
# Function for simulating driver check-ins in Delhi area (adjust coordinates)
def simulate_driver_checkins(num_drivers):
    drivers = []
    for i in range(num_drivers):
        longitude = 77.19  + random.uniform(-0.02, 0.02)  # Random around Delhi longitude
        latitude = 28.54  + random.uniform(-0.02, 0.02)   # Random around Delhi latitude
        driver_id = f"driver{i}"
        redis_client.geoadd("available_drivers", (longitude, latitude, driver_id))
        drivers.append({'driver_id': driver_id, 'longitude': longitude, 'latitude': latitude})
    return drivers

In [500]:
# Function for simulating rider queries 
def simulate_rider_queries(num_riders):
    riders = []
    for i in range(num_riders):
        rider_id = f"rider{i}"
        user_longitude = 77.19 + random.uniform(-0.02, 0.02)  # Random around Delhi longitude
        user_latitude = 28.54 + random.uniform(-0.02, 0.02)  # Random around Delhi latitude
        radius = 2  # km

        nearby_drivers = redis_client.georadius(
            "available_drivers", user_longitude, user_latitude, radius, unit='km', withdist=True
        )

        # Sort nearby_drivers by distance (ascending order)
        nearby_drivers.sort(key=lambda x: x[1])  # Sort by distance (second element in tuple)

        new_rider = {
            'rider_id': rider_id,
            'longitude': user_longitude,
            'latitude': user_latitude,
            'nearby_drivers': nearby_drivers
        }
        print(new_rider)
        riders.append(new_rider)

    return riders

In [501]:
# Simulate check-ins
drivers = simulate_driver_checkins(10)

In [502]:
# Simulate rider queries
riders = simulate_rider_queries(5)


{'rider_id': 'rider0', 'longitude': 77.18720047798074, 'latitude': 28.54470345867404, 'nearby_drivers': [[b'driver8', 0.4377], [b'driver6', 0.4642], [b'driver9', 0.8245], [b'driver3', 1.5322], [b'driver2', 1.6348]]}
{'rider_id': 'rider1', 'longitude': 77.17465640328827, 'latitude': 28.539464125727577, 'nearby_drivers': [[b'driver0', 0.8548], [b'driver8', 1.5037], [b'driver6', 1.7684], [b'driver2', 1.777], [b'driver5', 1.9779]]}
{'rider_id': 'rider2', 'longitude': 77.17361681546197, 'latitude': 28.55912842402336, 'nearby_drivers': [[b'driver7', 0.17], [b'driver1', 0.2795], [b'driver2', 0.4625], [b'driver6', 1.9907]]}
{'rider_id': 'rider3', 'longitude': 77.20686695183448, 'latitude': 28.538962112929887, 'nearby_drivers': [[b'driver3', 1.3165], [b'driver9', 1.6095], [b'driver8', 1.6859], [b'driver6', 1.9624]]}
{'rider_id': 'rider4', 'longitude': 77.19996728424096, 'latitude': 28.550492673884538, 'nearby_drivers': [[b'driver9', 0.5826], [b'driver4', 0.8219], [b'driver6', 1.0283], [b'driver

In [503]:
df_drivers = gpd.GeoDataFrame(drivers, geometry=[Point(row['longitude'], row['latitude']) for row in drivers])
df_riders = gpd.GeoDataFrame(riders, geometry=[Point(row['longitude'], row['latitude']) for row in riders])


In [543]:
# Create a map
center_location = (df_drivers.geometry.y.mean(), df_drivers.geometry.x.mean())  # Average location
map = folium.Map(location=center_location, zoom_start=13)

map

In [544]:
# Add driver markers with Taxi emoji 🚕
for _, row in df_drivers.iterrows():
    driver_emoji = "🚕"
    driver_id = row["driver_id"]
    folium.Marker(location=[row.geometry.y, row.geometry.x], icon=folium.DivIcon(
        icon_size=(25,25),
        icon_anchor=(0,0),
        html=f"""
            <div style="font-size: 30px;">{driver_emoji}</div>
            <div style="font-size: 12px; font-weight: bold;">{driver_id}</div>
        """
    )
        ).add_to(map)


# Add rider markers with 🙋‍♂️ or 🙋‍♀️ 
for _, row in df_riders.iterrows():
    # because diversity is important
    rider_emoji = random.choice(["🙋‍♂️", "🙋‍♀️"])
    rider_id = row["rider_id"] 

    folium.Marker(location=[row.geometry.y, row.geometry.x],icon=folium.DivIcon(
        icon_size=(25,25),
        icon_anchor=(0,0),
       html=f"""
            <div style="font-size: 30px;">{rider_emoji}</div>
            <div style="font-size: 12px; font-weight: bold;">{rider_id}</div>
        """
    )
        ).add_to(map)

map

In [545]:
def draw_match_line(map, rider_location, driver_location):
    # Extract coordinates from Point objects (if needed)
    if isinstance(rider_location, Point):
        rider_coords = [rider_location.y, rider_location.x]
    else:
        rider_coords = rider_location

    driver_coords = driver_location

    # Calculate distance (in kilometers)
    distance = geopy.distance.distance(rider_coords, driver_coords).km

    # Create polyline with distance label
    folium.PolyLine(
        locations=[rider_coords, driver_coords], color="green", weight=4, opacity=1, dashArray="5, 5"
    ).add_to(map)


In [546]:
# Simulation time!
# In an ideal world when nearest driver always accept your ride.
# Match riders to drivers and draw lines, with logs
matched_drivers = set()

for i, rider in df_riders.iterrows():
    rider_id = rider["rider_id"]
    print("-"*10)
    print(f"Time: t{i}")
    print("-"*10)
    print(f"{rider_id} looking for a ride...")

    nearby_drivers = rider["nearby_drivers"]
    nearby_drivers.sort(key=lambda x: x[1])

    print(f"Drivers found nearby: {nearby_drivers}")

    found_driver = False
    for driver_info, distance in nearby_drivers:
        driver_id = driver_info.decode()
        if driver_id not in matched_drivers:
            matched_drivers.add(driver_id)
            found_driver = True
            print(f"Ride matched to: {driver_id}")
            break

    # If no available driver is found, draw a red circle around the rider
    if not found_driver:
        print(f"No driver found for {rider_id}!")
        folium.Circle(
            location=[rider.geometry.y, rider.geometry.x],
            radius=100,  # Adjust radius as needed
            color="red",
            fill=True,
            fill_color="red",
            fill_opacity=0.5,
        ).add_to(map)
        continue

    if driver_id in matched_drivers:
        driver_row = df_drivers[df_drivers["driver_id"] == driver_id].iloc[0]
        driver_location = [driver_row.geometry.y, driver_row.geometry.x]  # Extract coordinates
        draw_match_line(map, rider.geometry, driver_location)
        print("Ride is On!\n")

# Display the map with lines
map

----------
Time: t0
----------
rider0 looking for a ride...
Drivers found nearby: [[b'driver8', 0.4377], [b'driver6', 0.4642], [b'driver9', 0.8245], [b'driver3', 1.5322], [b'driver2', 1.6348]]
Ride matched to: driver8
Ride is On!

----------
Time: t1
----------
rider1 looking for a ride...
Drivers found nearby: [[b'driver0', 0.8548], [b'driver8', 1.5037], [b'driver6', 1.7684], [b'driver2', 1.777], [b'driver5', 1.9779]]
Ride matched to: driver0
Ride is On!

----------
Time: t2
----------
rider2 looking for a ride...
Drivers found nearby: [[b'driver7', 0.17], [b'driver1', 0.2795], [b'driver2', 0.4625], [b'driver6', 1.9907]]
Ride matched to: driver7
Ride is On!

----------
Time: t3
----------
rider3 looking for a ride...
Drivers found nearby: [[b'driver3', 1.3165], [b'driver9', 1.6095], [b'driver8', 1.6859], [b'driver6', 1.9624]]
Ride matched to: driver3
Ride is On!

----------
Time: t4
----------
rider4 looking for a ride...
Drivers found nearby: [[b'driver9', 0.5826], [b'driver4', 0.821